# Grant Anna Access to Finance Products Table

This notebook demonstrates how to grant Anna (user 2) specific access to the `demo.finance.products` table using Cedar policies.

**Scenario**: Anna needs read access to the products table but not to other tables in the finance schema.

## 1. Add Cedar Policy for Anna

Anna's user ID in Keycloak is `oidc~d223d88c-85b6-4859-b5c5-27f3825e47f6`.

Grant her full access to the `demo` warehouse using the simplest pattern - all actions.

In [ ]:
# View current policies and save the original state
with open('/home/jovyan/policies/policies.cedar', 'r') as f:
    original_policies = f.read()

print(original_policies)
print(f"\n💾 Original policies saved to variable")

# Anna's user ID from Keycloak
anna_id = "oidc~d223d88c-85b6-4859-b5c5-27f3825e47f6"

# Cedar policy to grant Anna read access only to finance.products table
# Need permissions at ALL hierarchy levels: Project -> Warehouse -> Namespace -> Table
anna_policy = f'''
permit (
    principal == Lakekeeper::User::"{anna_id}",
    action in [
        Lakekeeper::Action::"ProjectDescribeActions",
        Lakekeeper::Action::"WarehouseDescribeActions",
        Lakekeeper::Action::"NamespaceDescribeActions",
        Lakekeeper::Action::"TableSelectActions"
    ],
    resource
)
when {{
    (resource is Lakekeeper::Project) ||
    (resource is Lakekeeper::Warehouse && resource.name == "demo") ||
    (resource is Lakekeeper::Namespace && 
     resource.name == "finance" && 
     resource.warehouse.name == "demo") ||
    (resource is Lakekeeper::Table && 
     resource.name == "products" &&
     resource.namespace.name == "finance" &&
     resource.warehouse.name == "demo")
}};
'''

# Append to policies.cedar
with open('/home/jovyan/policies/policies.cedar', 'a') as f:
    f.write(anna_policy)

print("✅ Cedar policy added for Anna!")
print("\nPolicy content:")
print(anna_policy)

## 4. Remove Anna's Policy

To revoke Anna's access, remove her policy from the file:

In [ ]:
# Restore the original policies (before Anna was added)
with open('/home/jovyan/policies/policies.cedar', 'w') as f:
    f.write(original_policies)

print("🗑️  Anna's policy removed!")
print("\nRestored to original policies:")
print(original_policies)